## 1. Tải Dữ Liệu từ CSV

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModel
import numpy as np
import torch
from datasets import load_dataset
import torch.nn as nn
import os
from typing import List
from tqdm import tqdm


# os.environ["CUDA_VISIBLE_DEVICES"] = "1" ## Setup CUDA GPU 1



In [3]:
import torch

# Kiểm tra GPU khả dụng
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPU found.")


CUDA available: True
Number of GPUs: 1
GPU 0: Tesla T4


In [4]:
import os
import torch

def select_gpu():
    """
    Kiểm tra GPU khả dụng và tự động chọn GPU phù hợp.
    """
    if torch.cuda.is_available():
        num_gpus = torch.cuda.device_count()
        print(f"Number of GPUs available: {num_gpus}")

        # Duyệt qua các GPU khả dụng để tìm GPU ít sử dụng nhất
        available_gpus = [torch.cuda.get_device_name(i) for i in range(num_gpus)]
        print("Available GPUs:", available_gpus)

        for i in range(num_gpus):
            try:
                # Đặt GPU
                os.environ["CUDA_VISIBLE_DEVICES"] = str(i)
                device = torch.device(f"cuda:{i}")
                torch.cuda.set_device(device)
                print(f"Using GPU: {torch.cuda.get_device_name(device.index)}")
                return device
            except Exception as e:
                print(f"GPU {i} is not suitable: {e}")

        print("No suitable GPU found. Falling back to CPU.")
        return torch.device("cpu")
    else:
        print("No GPUs available. Using CPU.")
        return torch.device("cpu")

# Tự động chọn GPU hoặc CPU
device = select_gpu()

# Kiểm tra lại thiết bị đang sử dụng
print(f"Final selected device: {device}")


Number of GPUs available: 1
Available GPUs: ['Tesla T4']
Using GPU: Tesla T4
Final selected device: cuda:0


In [5]:

class BERTIntentClassification(nn.Module):


    def __init__(self, model_name="bert-base-uncased", num_classes=10, dropout_rate=0.1, cache_dir = "huggingface"):
        super(BERTIntentClassification, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name, cache_dir = cache_dir)
        # Get BERT hidden size
        hidden_size = self.bert.config.hidden_size
        self.ffnn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_classes)
        )


    def freeze_bert(self):
        for param in self.bert.parameters():
            param.requires_grad = False


    def get_pooling(self, hidden_state, attention_mask):
        """
        Get mean pooled representation from BERT hidden states
        Args:
            hidden_state: BERT output containing hidden states
        Returns:
            pooled_output: Mean pooled representation of the sequence
        """
        # Get last hidden state
        last_hidden_state = hidden_state.last_hidden_state  # Shape: [batch_size, seq_len, hidden_size]

        if attention_mask is not None:
            # Expand attention mask to match hidden state dimensions
            attention_mask = attention_mask.unsqueeze(-1)  # [batch_size, seq_len, 1]

            # Mask out padding tokens
            masked_hidden = last_hidden_state * attention_mask

            # Calculate mean (sum / number of actual tokens)
            sum_hidden = torch.sum(masked_hidden, dim=1)  # [batch_size, hidden_size]
            count_tokens = torch.sum(attention_mask, dim=1)  # [batch_size, 1]
            pooled_output = sum_hidden / count_tokens
        else:
            # If no attention mask, simply take mean of all tokens
            pooled_output = torch.mean(last_hidden_state, dim=1)

        return pooled_output


    def forward(self, input_ids, attention_mask, **kwargs):
        """
        Forward pass of the model
        Args:
            input_ids: Input token IDs
            attention_mask: Attention mask for padding
        Returns:
            logits: Raw logits for each class
        """
        # Get BERT hidden states
        hidden_state = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

        # Get pooled representation
        hidden_state_pooling = self.get_pooling(hidden_state=hidden_state, attention_mask=attention_mask)

        # Pass through FFNN classifier
        logits = self.ffnn(hidden_state_pooling)

        return logits


In [6]:
class TrainerCustom(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        # Sử dụng nn.CrossEntropyLoss() thay vì nn.CrossEntropy
        cross_entropy_loss = nn.CrossEntropyLoss()

        # Chạy mô hình và nhận đầu ra (logits)
        outputs = model(**inputs)

        # Đảm bảo lấy logits từ outputs (mô hình trả về tuple, lấy phần tử đầu tiên là logits)
        logits = outputs

        # Tính toán loss
        loss = cross_entropy_loss(logits, labels)

        # Trả về loss và outputs nếu cần
        return (loss, outputs) if return_outputs else loss


# 1. Load Dataset and with Dataloader


**Cách kết hợp question và answer:**

1. **Ghép nối trực tiếp:** Bạn có thể kết hợp câu hỏi và câu trả lời thành một chuỗi duy nhất, sử dụng một ký tự đặc biệt hoặc dấu phân cách để tách biệt chúng. Ví dụ:

   ```python
   combined_text = question + " [SEP] " + answer
   ```

   Trong đó, `[SEP]` là một token đặc biệt thường được sử dụng trong các mô hình như BERT để phân tách các đoạn văn bản khác nhau.

2. **Sử dụng token đặc biệt:** Một số mô hình hỗ trợ các token đặc biệt để đánh dấu bắt đầu và kết thúc của câu hỏi và câu trả lời. Ví dụ:

   ```python
   combined_text = "[CLS] " + question + " [SEP] " + answer + " [SEP]"
   ```

   - `[CLS]`: Token đánh dấu bắt đầu chuỗi (thường dùng trong BERT).
   - `[SEP]`: Token phân tách giữa các phần của chuỗi.



Ví dụ thực tế về chuỗi chuẩn:

Một câu/đoạn duy nhất:
```
[CLS] This is the first sentence. [SEP]
```
Hai câu/đoạn (ví dụ: câu hỏi và trả lời):
```
[CLS] What is your name? [SEP] My name is John. [SEP]
```
Nhiều câu/đoạn (3 đoạn):
```
[CLS] Question 1 [SEP] Answer 1 [SEP] Extra information [SEP]
```

```
                          input_ids  intent
0  [CLS] Cậu có muốn tiếp tục không? [SEP]  silence
1                          [CLS] [SEP]  silence

```

In [12]:
import pandas as pd
from datasets import Dataset

def combine_with_special_tokens(row, text_columns, cls_token="[CLS]", sep_token="[SEP]"):
    """
    Thêm các token đặc biệt vào chuỗi kết hợp từ các cột văn bản.

    Args:
        row (pd.Series): Dòng dữ liệu từ DataFrame.
        text_columns (list): Danh sách các cột văn bản cần kết hợp.
        cls_token (str): Token bắt đầu câu.
        sep_token (str): Token phân cách.

    Returns:
        str: Chuỗi văn bản đã thêm token đặc biệt.
    """
    tokens = [cls_token]  # Thêm [CLS] đầu tiên

    # Thêm nội dung từ các cột văn bản
    for col in text_columns:
        if pd.notna(row[col]) and row[col].strip():  # Kiểm tra không rỗng
            tokens.append(row[col].strip())
            tokens.append(sep_token)  # Thêm [SEP] sau mỗi đoạn

    # Nếu không có nội dung nào được thêm, chỉ giữ lại [CLS] và [SEP]
    if len(tokens) == 1:
        tokens.append(sep_token)

    return " ".join(tokens)

def load_xlsx_dataset(xlsx_path, text_columns, label_column, cls_token="[CLS]", sep_token="[SEP]"):
    """
    Tải dataset từ file Excel (.xlsx) và xử lý dữ liệu.

    Args:
        xlsx_path (str): Đường dẫn đến file .xlsx.
        text_columns (list): Danh sách các cột cần ghép để tạo văn bản đầu vào.
        label_column (str): Tên cột chứa nhãn.
        cls_token (str): Token bắt đầu câu.
        sep_token (str): Token phân cách.

    Returns:
        Dataset: Tập dữ liệu đã xử lý.
    """
    # Đọc file Excel bằng pandas
    df = pd.read_excel(xlsx_path)

    # Kiểm tra các cột cần thiết
    for col in text_columns + [label_column]:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

    # Ghép các cột text lại thành một chuỗi duy nhất với token đặc biệt
    df["input_ids"] = df.apply(lambda row: combine_with_special_tokens(row, text_columns, cls_token, sep_token), axis=1)

    # Đổi tên cột nhãn
    df = df.rename(columns={label_column: "label"})

    # Chuyển đổi DataFrame thành Dataset
    dataset = Dataset.from_pandas(df[["input_ids", "label"]])

    return dataset

# Sử dụng hàm
xlsx_path = "/content/processed_data_example_v2_100data.xlsx"  # Đường dẫn file Excel
text_columns = ["robot", "user_answer"]  # Các cột cần ghép
label_column = "user_intent"  # Cột chứa nhãn

# Tải dataset từ Excel
dataset = load_xlsx_dataset(xlsx_path, text_columns, label_column)

# Kiểm tra dữ liệu
print(dataset)

# Lấy 10 mẫu đầu tiên
sample_dataset = dataset.select(range(20))
print(sample_dataset)

# In thử một hàng
print("First row in dataset:")
print(sample_dataset[0])
print(sample_dataset[11])


Dataset({
    features: ['input_ids', 'label'],
    num_rows: 120
})
Dataset({
    features: ['input_ids', 'label'],
    num_rows: 20
})
First row in dataset:
{'input_ids': "[CLS] Cậu có thể kể tên một số hành động bắt đầu bằng từ 'play' không? [SEP] Tớ có thể nói 'play football' và 'play basketball'. [SEP]", 'label': 'intent_positive'}
{'input_ids': '[CLS] Cậu có muốn tiếp tục không? [SEP]', 'label': 'silence'}


In [13]:
def check_invalid_samples(dataset):
    invalid_samples = []
    for idx, sample in enumerate(dataset):
        if not isinstance(sample["input_ids"], str) or sample["input_ids"].strip() == "":
            invalid_samples.append((idx, sample))
    return invalid_samples

# Kiểm tra dữ liệu không hợp lệ
invalid_samples = check_invalid_samples(dataset)
print("\n===== Invalid Samples =====")
print(invalid_samples)



===== Invalid Samples =====
[]


In [14]:
# Tự động phát hiện nhãn và tạo ánh xạ nhãn
def create_label_mapping(dataset_list):
    """
    Tự động phát hiện tất cả các nhãn từ danh sách dataset và ánh xạ chúng thành số nguyên.
    """
    all_labels = set()
    for dataset in dataset_list:
        all_labels.update(dataset["label"])  # Tập hợp tất cả các nhãn từ dataset

    label_to_int = {label: idx for idx, label in enumerate(sorted(all_labels))}
    print(f"Ánh xạ nhãn: {label_to_int}")
    return label_to_int

# Hàm chuyển đổi nhãn
def preprocess_labels(example, label_to_int):
    example["label"] = label_to_int.get(example["label"], -1)  # Gán -1 cho nhãn không hợp lệ
    return example

# Tạo ánh xạ nhãn
label_mapping = create_label_mapping([dataset])

# Áp dụng chuyển đổi nhãn
dataset = dataset.map(lambda example: preprocess_labels(example, label_mapping))

# Kiểm tra kết quả
print(dataset)

# Truy cập mẫu cụ thể
sample_dataset = dataset.select(range(10))  # Lấy 10 mẫu đầu tiên
print(sample_dataset)

# In thử 1 hàng trong sample_dataset
print("First row in sample_dataset:")
print(sample_dataset[1])

Ánh xạ nhãn: {'intent_fallback': 0, 'intent_learn_more': 1, 'intent_negative': 2, 'intent_neutral': 3, 'intent_positive': 4, 'silence': 5}


Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'label'],
    num_rows: 120
})
Dataset({
    features: ['input_ids', 'label'],
    num_rows: 10
})
First row in sample_dataset:
{'input_ids': "[CLS] Cậu có biết thêm từ nào khác không? [SEP] Tớ biết 'play games' nữa. [SEP]", 'label': 4}


In [15]:
def split_dataset(dataset, test_size=0.2, seed=42):
    """
    Chia dataset thành tập train và test.

    Args:
        dataset (Dataset): Tập dữ liệu đầy đủ.
        test_size (float): Tỷ lệ dữ liệu test (0.0 - 1.0).
        seed (int): Seed để chia dữ liệu ngẫu nhiên.

    Returns:
        tuple: (train_dataset, test_dataset) - Tập train và test.
    """
    if not (0.0 < test_size < 1.0):
        raise ValueError("test_size phải nằm trong khoảng (0.0, 1.0)")
    if len(dataset) < 2:
        raise ValueError("Dataset phải có ít nhất 2 mẫu để chia.")

    train_test_split = dataset.train_test_split(test_size=test_size, seed=seed)
    print(f"Chia dataset: {len(train_test_split['train'])} mẫu train, {len(train_test_split['test'])} mẫu test")
    return train_test_split["train"], train_test_split["test"]

# Chia dataset
train_dataset, test_dataset = split_dataset(dataset, test_size=0.3)

# Kiểm tra dữ liệu
print("Train dataset:", train_dataset)
print("Test dataset:", test_dataset)

# Truy cập mẫu cụ thể
sample_train_dataset = train_dataset.select(range(8))  # Lấy 10 mẫu đầu tiên từ train
sample_test_dataset = test_dataset.select(range(5))    # Lấy 10 mẫu đầu tiên từ test

print("Sample train dataset:", sample_train_dataset)
print("Sample test dataset:", sample_test_dataset)

Chia dataset: 84 mẫu train, 36 mẫu test
Train dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 84
})
Test dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 36
})
Sample train dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 8
})
Sample test dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 5
})


# 2. Tokenizer

In [16]:
# Calculate the number of unique labels
print(label_mapping)
number_label = len(label_mapping)
print("Number of unique labels:", number_label)

{'intent_fallback': 0, 'intent_learn_more': 1, 'intent_negative': 2, 'intent_neutral': 3, 'intent_positive': 4, 'silence': 5}
Number of unique labels: 6


In [17]:
# Bước 2: Chuẩn bị tokenizer và token hóa dữ liệu
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir = "huggingface")
model = BERTIntentClassification(
    model_name=model_name,
    num_classes=6
)
model.freeze_bert() # Froze Layer BERT
max_seq_length = 512


def collate_fn(features):
    inputs = []
    labels = []
    for element in features:
        inputs.append(element.get("input_ids"))
        labels.append(element.get("label"))

    labels = torch.tensor(labels, dtype=torch.long)

    token_inputs = tokenizer(
        inputs,
        add_special_tokens=True,
        truncation=True,
        padding=True,
        max_length=max_seq_length,
        return_overflowing_tokens=False,
        return_length=False,
        return_tensors="pt",
    )
    token_inputs.update({
        "labels": labels,
    })
    return token_inputs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

# 3. Train Model

## 3.1 Log Wandb

In [18]:
!pip install --upgrade wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 86.4 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.19.1
    Uninstalling wandb-0.19.1:
      Successfully uninstalled wandb-0.19.1


In [19]:
!pip install python-dotenv

In [20]:
from dotenv import load_dotenv
import os

# Load biến môi trường từ file .env
load_dotenv()

# Lấy key từ biến môi trường
wandb_api_key = os.getenv("WANDB_API_KEY")
print(wandb_api_key[:5])

c8767


In [21]:
import wandb
import os

# Lấy API key từ biến môi trường và đăng nhập
wandb.login(key=os.getenv("WANDB_API_KEY"))


wandb: Currently logged in as: doanngoccuong (doanngoccuong_nh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Cách thiết lập thông qua TrainingArguments
Khi sử dụng Trainer, bạn có thể đặt tên dự án trực tiếp trong TrainingArguments bằng cách sử dụng tham số report_to và run_name. Tuy nhiên, để đặt project, bạn cần khởi tạo một phiên wandb trước hoặc truyền cấu hình này thông qua wandb.init().

Điều chỉnh TrainingArguments:
```python
training_args = TrainingArguments(
    output_dir="./results_",          # Thư mục lưu kết quả
    eval_strategy="epoch",           # Đánh giá sau mỗi epoch
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",            # Thư mục lưu log
    logging_strategy="steps",        # Log theo steps
    logging_steps=10,                # Log sau mỗi 10 bước
    save_strategy="epoch",           # Lưu checkpoint sau mỗi epoch
    save_total_limit=3,              # Lưu tối đa 3 checkpoint
    report_to="wandb",               # Báo cáo log tới wandb
    run_name="bert_run_1"            # Tên phiên chạy trên wandb
)
```

## 3.2 Train

### Ver 1.2.3

Thui, ko lưu local nữa, lưu tất trên wandb đi.
- Với best model: lưu lên wandb khi loss giảm và đã sau 10 epochs  
(Lưu Best Model ngay khi eval_loss giảm ở local, sau 10 epochs thì đồng bộ cái best lên wandb, sau đó xoá các file best ở local).
Chỉ đồng bộ lên WandB mỗi 10 epochs.)
- Với last model: lưu lên wandb sau mỗi 10 epochs. (lưu local trước -> đồng bộ lên wandb sẽ xoá file local)
+, Trong quá trình lưu thì việc training vẫn diễn ra Parallel

đều lưu đầy đủ toàn bộ tham số để có thể train thêm từ cả ở best model và last model

In [26]:
from concurrent.futures import ThreadPoolExecutor
import wandb
import os
import shutil
import time

class TrainerCustom(Trainer):
    def __init__(self, *args, save_every_n_epochs=10, **kwargs):
        super().__init__(*args, **kwargs)
        if torch.cuda.is_available():
            print(f"Trainer is running on GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
        else:
            print("Trainer is running on CPU.")

        self.best_eval_loss = float("inf")  # Giá trị loss tốt nhất ban đầu
        self.save_every_n_epochs = save_every_n_epochs  # Tần suất lưu lên WandB
        self.best_model_info = {"epoch": None, "loss": None}
        self.last_saved_epoch = 0  # Epoch cuối cùng đã lưu Best Model và Last Model
        self.executor = ThreadPoolExecutor(max_workers=3)  # Cho phép tối đa 2 luồng song song

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """

        # # Kiểm tra thiết bị của mô hình và dữ liệu
        # print("Model device:", next(model.parameters()).device)
        # print("Input device:", inputs["input_ids"].device)
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        # Sử dụng nn.CrossEntropyLoss() thay vì nn.CrossEntropy
        cross_entropy_loss = nn.CrossEntropyLoss()

        # Chạy mô hình và nhận đầu ra (logits)
        outputs = model(**inputs)

        # Đảm bảo lấy logits từ outputs (mô hình trả về tuple, lấy phần tử đầu tiên là logits)
        logits = outputs

        if labels is None:
            print("Labels are None during compute_loss.")
        if logits is None:
            print("Logits are None during compute_loss.")

        # Tính toán loss
        loss = cross_entropy_loss(logits, labels)

        # Trả về loss và outputs nếu cần
        return (loss, outputs) if return_outputs else loss

    def async_save_model(self, model_dir, artifact_name, metadata=None):
        """
        Lưu mô hình vào local và đồng bộ lên WandB trong luồng song song.
        """
        def save():
            start_time = time.time()
            try:
                # Xóa tất cả các thư mục tmp_best_model_ trước đó
                for folder in os.listdir("."):
                    if folder.startswith("tmp_best_model_epoch_") and folder != model_dir:
                        shutil.rmtree(folder, ignore_errors=True)
                        print(f"Removed old temporary directory: {folder}")

                # Lưu mô hình vào thư mục tạm
                self.save_model(model_dir)

                # Đồng bộ lên WandB
                artifact = wandb.Artifact(artifact_name, type="model")
                artifact.add_dir(model_dir)
                if metadata:
                    artifact.metadata = metadata
                wandb.log_artifact(artifact)
            except Exception as e:
                print(f"Error during saving or syncing model {artifact_name}: {e}")
            finally:
                # Xóa thư mục tạm hiện tại sau khi đồng bộ
                try:
                    shutil.rmtree(model_dir, ignore_errors=True)
                    print(f"Successfully removed temporary directory: {model_dir}")
                except Exception as e:
                    print(f"Error removing temporary directory {model_dir}: {e}")

            elapsed_time = time.time() - start_time
            print(f"Model saved and uploaded to WandB: {artifact_name} in {elapsed_time:.2f} seconds")

        self.executor.submit(save)




    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix: str = "eval"):
        metrics = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
        eval_loss = metrics.get("eval_loss")

        # Cập nhật Best Model nếu eval_loss giảm
        # Lưu Best Model ngay khi eval_loss giảm (local).
        # Đồng bộ wandb ngay

        if eval_loss is not None and eval_loss < self.best_eval_loss:
            print(f"New best eval_loss: {eval_loss}")
            self.best_eval_loss = eval_loss
            self.best_model_info = {"epoch": self.state.epoch, "loss": eval_loss}

            # Log thông tin Best Model lên WandB
            wandb.log({
                "best_eval_loss": self.best_eval_loss,
                "best_model_epoch": self.best_model_info.get("epoch", -1)
            })

            # Lưu Best Model vào thư mục tạm (local)
            best_model_dir = f"./tmp_best_model_epoch_{int(self.state.epoch)}"
            self.save_model(best_model_dir)

            # Đồng bộ lên WandB ngay
            artifact_name = f"best_model_epoch_{int(self.state.epoch)}"
            self.async_save_model(best_model_dir, artifact_name, self.best_model_info)

        return metrics

    # def save_last_model(self):
    #     """
    #     Lưu Last Model lên WandB sau mỗi N epochs.
    #     """
    #     if int(self.state.epoch) % self.save_every_n_epochs == 0 and int(self.state.epoch) != self.last_saved_epoch:
    #         print(f"Saving Last Model at epoch {self.state.epoch} to WandB...")
    #         last_model_dir = f"./tmp_last_model_epoch_{int(self.state.epoch)}"
    #         artifact_name = f"last_model_epoch_{int(self.state.epoch)}"
    #         self.async_save_model(last_model_dir, artifact_name)

    #         # Log thông tin Last Model lên WandB
    #         wandb.log({
    #             "last_model_epoch": self.state.epoch
    #         })

    #         # Cập nhật epoch cuối cùng đã lưu
    #         self.last_saved_epoch = int(self.state.epoch)

    def save_last_model(self):
        """
        Lưu Last Model (bao gồm trạng thái optimizer, scheduler) lên WandB sau mỗi N epochs.
        """
        if int(self.state.epoch) % self.save_every_n_epochs == 0 and int(self.state.epoch) != self.last_saved_epoch:
            print(f"Saving Last Model at epoch {self.state.epoch} to WandB...")

            # Thư mục tạm lưu checkpoint
            last_model_dir = f"./tmp_last_model_epoch_{int(self.state.epoch)}"
            os.makedirs(last_model_dir, exist_ok=True)

            # Lưu đầy đủ trạng thái mô hình (checkpoint)
            self.save_model(last_model_dir)

            # Đường dẫn tệp trainer_state.json
            trainer_state_path = os.path.join(last_model_dir, "trainer_state.json")
            self.state.save_to_json(trainer_state_path)  # Lưu trạng thái trainer

            # Đồng bộ checkpoint lên WandB
            artifact_name = f"last_model_epoch_{int(self.state.epoch)}"
            metadata = {
                "epoch": int(self.state.epoch),
                "last_eval_loss": self.state.best_metric if hasattr(self.state, "best_metric") else "N/A",
            }
            self.async_save_model(last_model_dir, artifact_name, metadata)

            # Cập nhật epoch cuối cùng đã lưu
            self.last_saved_epoch = int(self.state.epoch)


    def train(self, *args, **kwargs):
        result = super().train(*args, **kwargs)

        # Sau mỗi epoch, lưu Last Model lên WandB
        self.save_last_model()
        # Chờ tất cả các luồng lưu hoàn thành trước khi kết thúc
        self.executor.shutdown(wait=True)

        return result


# Bước 6: Cài đặt tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./result__s",          # Thư mục lưu kết quả
    eval_strategy="epoch",    # Đánh giá sau mỗi epoch
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=1,  # Ghi logs mỗi 500 bước huấn luyện
    save_strategy="no",          # Lưu trọng số sau mỗi epoch
    save_total_limit=3,
    label_names = ["labels"],
    report_to="wandb",
    run_name="bert_run_3"
)


import wandb

# Khởi tạo wandb
wandb.init(
    project="bert-intent-classification",  # Tên dự án
    name="bert_10000Data_1epoch",                     # Tên phiên chạy
    config={"gpu": torch.cuda.get_device_name(torch.cuda.current_device()) if torch.cuda.is_available() else "CPU"}
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
if torch.cuda.is_available():
    print(f"Trainer is running on GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("Trainer is running on CPU.")

trainer = TrainerCustom(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=collate_fn,
    save_every_n_epochs=10  # Lưu Best Model và Last Model mỗi 10 epochs
)

trainer.train()


wandb.finish()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Trainer is running on GPU: Tesla T4
Trainer is running on GPU: Tesla T4


Epoch,Training Loss,Validation Loss
1,1.829300,1.795009
2,1.712800,1.703926
3,1.627400,1.609709
4,1.555400,1.533921
5,1.485700,1.484958
6,1.445600,1.430269
7,1.320500,1.374877
8,1.254900,1.320930
9,1.209100,1.271355
10,1.151700,1.211655


New best eval_loss: 1.7950087785720825
Removed old temporary directory: tmp_best_model_epoch_1
New best eval_loss: 1.7039257287979126


wandb: Adding directory to artifact (./tmp_best_model_epoch_1)... 

...

Done. 13.1s
wandb: Adding directory to artifact (./tmp_best_model_epoch_16)... 

Successfully removed temporary directory: ./tmp_best_model_epoch_15
Model saved and uploaded to WandB: best_model_epoch_15 in 21.01 seconds
New best eval_loss: 0.7918722629547119
Removed old temporary directory: tmp_best_model_epoch_16
Removed old temporary directory: tmp_best_model_epoch_17


Done. 17.0s
wandb: Adding directory to artifact (./tmp_best_model_epoch_17)... 

Successfully removed temporary directory: ./tmp_best_model_epoch_16
Model saved and uploaded to WandB: best_model_epoch_16 in 26.08 seconds
Removed old temporary directory: tmp_best_model_epoch_18
Removed old temporary directory: tmp_best_model_epoch_17


KeyboardInterrupt: 